In [2]:
%matplotlib inline

import matplotlib
matplotlib.style.use('ggplot')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [34]:
from bs4 import BeautifulSoup
import requests

url = "https://marketdata.set.or.th/mkt/sectorquotation.do?sector=SET100&language=en&country=US"
page = requests.get(url)
data = page.content
soup = BeautifulSoup(data)
SET100 = {}

# print soup

for i,link in enumerate(soup.find_all('td', style="text-align: left;")):
    for sublink in link.find_all('a'):
        html = sublink.get('href')
        if html[:14] == "stockquotation":
            name = html.split("stockquotation.do?symbol=")[1].split("&sso")[0]
            SET100[name] = {}
            SET100[name]["PE"] = {}
            SET100[name]["EPS"] = {}
            SET100[name]["ProfitMargin"] = {}
        else:
            pass

SET100

{'AAV': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'ADVANC': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'AMATA': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'AOT': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'AP': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BA': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BANPU': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BBL': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BCH': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BCP': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BDMS': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BEAUTY': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BEC': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BEM': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BH': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BIG': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BLA': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BLAND': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'BTS': {'EPS': {}, 'PE': {}, 'ProfitMargin': {}},
 'CBG': {'EPS': {}, '